In [1]:
# reference: https://github.com/ellisdg/3DUnetCNN/
import numpy as np

import keras
from keras import backend as K
from keras.engine import Input, Model
from keras.layers import Conv3D, MaxPooling3D, UpSampling3D, Activation, BatchNormalization, PReLU, Deconvolution3D
from keras.layers.merge import concatenate
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, CSVLogger, LearningRateScheduler, ReduceLROnPlateau, EarlyStopping
K.set_image_data_format("channels_first")

from model.dice import *

def unet_model_3d(input_shape, pool_size=(2, 2, 2), initial_learning_rate=1e-4,
                  deconvolution=False, depth=1, n_base_filters=1, metrics=dice_coefficient,
                  batch_normalization=False, activation_name="sigmoid"):
    """
    Builds the 3D UNet Keras model.f
    :param metrics: List metrics to be calculated during model training (default is dice coefficient).
    :param n_base_filters: The number of filters that the first layer in the convolution network will have. Following
    layers will contain a multiple of this number. Lowering this number will likely reduce the amount of memory required
    to train the model.
    :param depth: indicates the depth of the U-shape for the model. The greater the depth, the more max pooling
    layers will be added to the model. Lowering the depth may reduce the amount of memory required for training.
    :param input_shape: Shape of the input data (n_chanels, x_size, y_size, z_size). The x, y, and z sizes must be
    divisible by the pool size to the power of the depth of the UNet, that is pool_size^depth = 16
    :param pool_size: Pool size for the max pooling operations.
    :param n_labels: Number of binary labels that the model is learning.
    :param initial_learning_rate: Initial learning rate for the model. This will be decayed during training.
    :param deconvolution: If set to True, will use transpose convolution(deconvolution) instead of up-sampling. This
    increases the amount memory required during training.
    :return: Untrained 3D UNet Model
    """
    inputs = Input(input_shape)
    current_layer = inputs
    levels = []

    # add levels with max pooling
    for layer_depth in range(depth):
        layer1 = create_convolution_block(input_layer=current_layer, n_filters=n_base_filters*(2**layer_depth),
                                          batch_normalization=batch_normalization)
        layer2 = create_convolution_block(input_layer=layer1, n_filters=n_base_filters*(2**layer_depth)*2,
                                          batch_normalization=batch_normalization)
        if layer_depth < depth - 1:
            current_layer = MaxPooling3D(pool_size=pool_size)(layer2)
            levels.append([layer1, layer2, current_layer])
        else:
            current_layer = layer2
            levels.append([layer1, layer2])

    # add levels with up-convolution or up-sampling
    for layer_depth in range(depth-2, -1, -1):
        up_convolution = get_up_convolution(pool_size=pool_size, deconvolution=deconvolution,
                                            n_filters=current_layer._keras_shape[1])(current_layer)
        concat = concatenate([up_convolution, levels[layer_depth][1]], axis=1)
        current_layer = create_convolution_block(n_filters=levels[layer_depth][1]._keras_shape[1],
                                                 input_layer=concat, batch_normalization=batch_normalization)
        current_layer = create_convolution_block(n_filters=levels[layer_depth][1]._keras_shape[1],
                                                 input_layer=current_layer,
                                                 batch_normalization=batch_normalization)

    # number of labels: 1
    final_convolution = Conv3D(1, (1, 1, 1))(current_layer)
    act = Activation(activation_name)(final_convolution)
    model = Model(inputs=inputs, outputs=act)

    if not isinstance(metrics, list):
        metrics = [metrics]

    model.compile(optimizer=Adam(lr=initial_learning_rate), loss=dice_coefficient_loss, metrics=metrics)
#     model.compile(optimizer=Adam(lr=initial_learning_rate), loss=keras.losses.binary_crossentropy, metrics=metrics)
    return model


def create_convolution_block(input_layer, n_filters, batch_normalization=False, kernel=(3, 3, 3),
                             activation=None, padding='same', strides=(1, 1, 1), instance_normalization=False):
    """
    :param input_layer:
    :param n_filters:
    :param batch_normalization:
    :param kernel:
    :param activation: Keras activation layer to use. (default is 'relu')
    :param padding:
    :return:
    """
    layer = Conv3D(n_filters, kernel, padding=padding, strides=strides)(input_layer)
    if batch_normalization:
        layer = BatchNormalization(axis=1)(layer)
    elif instance_normalization:
        try:
            from keras_contrib.layers.normalization import InstanceNormalization
        except ImportError:
            raise ImportError("Install keras_contrib in order to use instance normalization."
                              "\nTry: pip install git+https://www.github.com/farizrahman4u/keras-contrib.git")
        layer = InstanceNormalization(axis=1)(layer)
    if activation is None:
        return Activation('relu')(layer)
    else:
        return activation()(layer)


def compute_level_output_shape(n_filters, depth, pool_size, image_shape):
    """
    Each level has a particular output shape based on the number of filters used in that level and the depth or number
    of max pooling operations that have been done on the data at that point.
    :param image_shape: shape of the 3d image.
    :param pool_size: the pool_size parameter used in the max pooling operation.
    :param n_filters: Number of filters used by the last node in a given level.
    :param depth: The number of levels down in the U-shaped model a given node is.
    :return: 5D vector of the shape of the output node
    """
    output_image_shape = np.asarray(np.divide(image_shape, np.power(pool_size, depth)), dtype=np.int32).tolist()
    return tuple([None, n_filters] + output_image_shape)


def get_up_convolution(n_filters, pool_size, kernel_size=(2, 2, 2), strides=(2, 2, 2), deconvolution=False):
    if deconvolution:
        return Deconvolution3D(filters=n_filters, kernel_size=kernel_size, strides=strides)
    else:
        return UpSampling3D(size=pool_size)


def get_callbacks(file_path, file, initial_learning_rate=0.0001, learning_rate_drop=0.5,
                  learning_rate_patience=50, verbosity=1, early_stopping_patience=None):

    check_point = ModelCheckpoint(file_path + '/fold' + file + 'weights-{epoch:02d}-{val_loss:.2f}.hdf5', save_best_only=False)
    csv_log = CSVLogger(file_path + '/training-log.csv', append=True)

    # potential problem of recude learning rate: https://github.com/keras-team/keras/issues/10924
    reduce = ReduceLROnPlateau(factor=learning_rate_drop, patience=learning_rate_patience, verbose=verbosity)
    if early_stopping_patience:
        early_stop = EarlyStopping(verbose=verbosity, patience=early_stopping_patience)
        return [check_point, csv_log, reduce, early_stop]
    else:
        return [check_point, csv_log, reduce]


def train_model(model, model_file, training_generator, validation_generator, steps_per_epoch, validation_steps,
            initial_learning_rate=0.001, learning_rate_drop=0.5, n_epochs=500,
            learning_rate_patience=20, early_stopping_patience=None):
    """
    Train a Keras model.
    :param early_stopping_patience: If set, training will end early if the validation loss does not improve after the
    specified number of epochs.
    :param learning_rate_patience: If learning_rate_epochs is not set, the learning rate will decrease if the validation
    loss does not improve after the specified number of epochs. (default is 20)
    :param model: Keras model that will be trained.
    :param model_file: Where to save the Keras model.
    :param training_generator: Generator that iterates through the training data.
    :param validation_generator: Generator that iterates through the validation data.
    :param steps_per_epoch: Number of batches that the training generator will provide during a given epoch.
    :param validation_steps: Number of batches that the validation generator will provide during a given epoch.
    :param initial_learning_rate: Learning rate at the beginning of training.
    :param learning_rate_drop: How much at which to the learning rate will decay.
    :param n_epochs: Total number of epochs to train the model.
    :return:
    """
    callbacks = get_callbacks(model_file,
                            initial_learning_rate=initial_learning_rate,
                            learning_rate_drop=learning_rate_drop,
                            learning_rate_patience=learning_rate_patience,
                            early_stopping_patience=early_stopping_patience)

    model.fit_generator(generator=training_generator,
                        steps_per_epoch=steps_per_epoch,
                        epochs=n_epochs,
                        validation_data=validation_generator,
                        validation_steps=validation_steps,
                        callbacks=callbacks)


Using TensorFlow backend.


In [2]:
import glob, os, random
import nibabel as nib
import numpy as np
import h5py

import matplotlib.pyplot as plt
from collections import defaultdict
from datetime import datetime

import ipywidgets as widgets
from ipywidgets import interact, interactive

from scipy import ndimage, misc

# directory: ./data/*/*.nii.gz
# there are different modalities that should be taken care of


class Data:
    def __init__(self, moda):
        self.moda = moda
        # ["FLAIR_preprocessed", "T1_preprocessed", "T2_preprocessed"]
        self.input = {m : defaultdict(list) for m in self.moda}
        self.target = defaultdict(list)
        self.kfold = None
        self.batch_size = None
        self.patch_size = None
        self.patch_gap = None
        self.patch_index = defaultdict(list)
        self.valid_index = {}
        # pre-set seed so that reconstruct can retrieve valid_index for final result
        random.seed(3000)
        
    def fetch_raw_data_from_file(self, modality, file_name):
        def fetch_file(modality):
            # could be "Consensus", "FLAIR_preprocessed"
            data = []
            root, sub_dir, _ = next(os.walk(os.getcwd() + '/data/'))
            for sub in sub_dir:
                data.append(os.path.join(root, sub + '/' + modality + '.nii.gz'))
            return data
        file = fetch_file(modality)
        raw_data = defaultdict(list)
        for i in range(len(file)):
            image = nib.load(file[i])
            raw_data[image.shape].append(image.get_fdata())
        with h5py.File(file_name, 'w') as f:
            for i in raw_data:
                f.create_dataset(str(i), data=raw_data[i])
        return raw_data
    
    def zero_pad(self, image, div=(32, 32, 32)):
        pad_size = [0, 0, 0]
        pad = False
        for i in range(len(image.shape)):
            remain = image.shape[i] % div[i]
            if remain != 0:
                pad = True
                pad_size[i] = (image.shape[i] // div[i] + 1) * div[i] - image.shape[i]
        if pad:
            # deal with odd number of padding
            pad0 = (pad_size[0]//2, pad_size[0] - pad_size[0]//2)
            pad1 = (pad_size[1]//2, pad_size[1] - pad_size[1]//2)
            pad2 = (pad_size[2]//2, pad_size[2] - pad_size[2]//2)
            # https://stackoverflow.com/questions/50008587/zero-padding-a-3d-numpy-array
            return np.pad(image, (pad0, pad1, pad2), 'constant')
        else:
            return image
    
    def check_file_exist(self, path, pattern):
        for m in self.moda:
            if not os.path.isfile(path + pattern + m + '.h5'):
                return False
        return True
    
    
    def fetch_target_data(self, target_path):
        self.fetch_raw_data_from_file('Consensus', target_path + 'target_data.h5')
        return self.load_target_data(target_path)
    
    def fetch_raw_data(self, raw_path):
        for m in self.moda:
            # different modality: same order of images
            self.fetch_raw_data_from_file(m, raw_path + 'raw_data_' + m + '.h5')
        return self.load_raw_data(raw_path)
    
    def load_target_data(self, target_path):
        target_file = h5py.File(target_path + 'target_data.h5', 'r')
        target_data = defaultdict(list)
        for i in target_file.keys():
            target_data[i] = target_file[i]
        return target_data, target_file
    
    def load_raw_data(self, raw_path):
        raw_file = []
        raw_data = {m : defaultdict(list) for m in self.moda}
        for m in self.moda:
            data = defaultdict(list)
            file = h5py.File(raw_path + 'raw_data_' + m + '.h5', 'r') 
            raw_file.append(file)
            for i in file.keys():
                # to get the matrix: self.data[i][:]
                # d.data[i][j][0], d.data[i][j][1]
                data[i] = file[i]
            raw_data[m] = data
        return raw_data, raw_file
    
        # raw_data:
        # {'FLAIR_preprocessed': defaultdict(list,
        #               {'(128, 224, 256)': <HDF5 dataset "(128, 224, 256)": shape (5, 128, 224, 256), type "<f8">,
        #                '(144, 512, 512)': <HDF5 dataset "(144, 512, 512)": shape (5, 144, 512, 512), type "<f8">,
        #                '(261, 336, 336)': <HDF5 dataset "(261, 336, 336)": shape (5, 261, 336, 336), type "<f8">}),
        #   'T1_preprocessed': defaultdict(list,
        #               {'(128, 224, 256)': <HDF5 dataset "(128, 224, 256)": shape (5, 128, 224, 256), type "<f8">,
        #                '(144, 512, 512)': <HDF5 dataset "(144, 512, 512)": shape (5, 144, 512, 512), type "<f8">,
        #                '(261, 336, 336)': <HDF5 dataset "(261, 336, 336)": shape (5, 261, 336, 336), type "<f8">}),
        #   'T2_preprocessed': defaultdict(list,
        #               {'(128, 224, 256)': <HDF5 dataset "(128, 224, 256)": shape (5, 128, 224, 256), type "<f8">,
        #                '(144, 512, 512)': <HDF5 dataset "(144, 512, 512)": shape (5, 144, 512, 512), type "<f8">,
        #                '(261, 336, 336)': <HDF5 dataset "(261, 336, 336)": shape (5, 261, 336, 336), type "<f8">})}
        # raw_file:
        #  [<HDF5 file "raw_data_FLAIR_preprocessed.h5" (mode r)>,
        #   <HDF5 file "raw_data_T1_preprocessed.h5" (mode r)>,
        #   <HDF5 file "raw_data_T2_preprocessed.h5" (mode r)>]

    def pad_target_data(self, patch_size, pad_path, target_path):
        # input
        target_data = None
        target_file = None
        if os.path.isfile(target_path + "target_data.h5"):
            target_data, target_file = self.load_target_data(target_path)
        else:
            target_data, target_file = self.fetch_target_data(target_path)
        
        pad_target_data = defaultdict(list)
        for i in target_data.keys():
            for j in range(target_data[i].shape[0]):
                img = self.zero_pad(target_data[i][j], patch_size)
                pad_target_data[img.shape].append(img)

        target_file.close()

        with h5py.File(pad_path + "pad_target_data.h5" , 'w') as f:
            f.create_dataset("patch_size", data=patch_size)
            for i in pad_target_data:
                f.create_dataset(str(i), data=pad_target_data[i])
        
        pad_file = h5py.File(pad_path + "pad_target_data.h5" , 'r')
        for i in pad_file.keys():
            if i == "patch_size":
                continue
            self.target[i] = pad_file[i][:]
        
    def pad_raw_data(self, patch_size, pad_path, raw_path):
        # input
        raw_data = None
        raw_file = None
        if self.check_file_exist(raw_path, "raw_data_"):
            raw_data, raw_file = self.load_raw_data(raw_path)
        else:
            raw_data, raw_file = self.fetch_raw_data(raw_path)
        
        pad_data = {m : defaultdict(list) for m in self.moda}
        for m in raw_data.keys():
            data = defaultdict(list)
            for i in raw_data[m].keys():
                for j in range(raw_data[m][i].shape[0]):
                    img = self.zero_pad(raw_data[m][i][j], patch_size)
                    data[img.shape].append(img)
            pad_data[m] = data
            
        for f in raw_file:
            f.close()

        for m in pad_data.keys():
            with h5py.File(pad_path + "pad_data_" + m + ".h5" , 'w') as f:
                f.create_dataset("patch_size", data=patch_size)
                for i in pad_data[m]:
                    f.create_dataset(str(i), data=pad_data[m][i])
        
        for m in pad_data.keys():
            pad_file = h5py.File(pad_path + "pad_data_" + m + ".h5" , 'r')
            for i in pad_file.keys():
                if i == "patch_size":
                    continue
                self.input[m][i] = pad_file[i][:]
        
    
    def load_data(self, patch_size=(32, 32, 32), 
                  pad_path="./model/h5df_data/", raw_path="./model/h5df_data/", target_path="./model/h5df_data/"):
                  # pad_path="./model/h5df_data/pad_data.h5", raw_path="./model/h5df_data/raw_data.h5"):
        padded = True
        if self.check_file_exist(raw_path, "pad_data_"):
            for m in self.moda:
                pad_file = h5py.File(pad_path + "pad_data_" + m + ".h5", 'r')
                if np.all(pad_file["patch_size"][:] == list(patch_size)):
                    # self.data = pad_file["pad_data"]
                    for i in pad_file.keys():
                        self.input[m][i] = pad_file[i][:]
                else:
                    padded = False
                    break
        else:
            padded = False
        if padded == False:
            self.pad_raw_data(patch_size, pad_path, raw_path)
        
        if os.path.isfile(target_path + "pad_target_data.h5"):
            pad_file = h5py.File(target_path + "pad_target_data.h5", 'r')
            if np.all(pad_file["patch_size"][:] == list(patch_size)):
                # self.data = pad_file["pad_data"]
                for i in pad_file.keys():
                    self.target[i] = pad_file[i][:]
            else:
                self.pad_target_data(patch_size, pad_path, target_path)
        else:
            self.pad_target_data(patch_size, pad_path, target_path)
            
    
    def show_image(self, images):
        # show image with [None, None, : ,: ,:] dimension
        def show_frame(id):
            length = len(images)
            for i in range(length):
                ax = plt.subplot(1, length, i+1)
                if (i == 0):
                    ax.set_title("Input")
                if (i == 1):
                    ax.set_title("Target")
                if (i == 2):
                    ax.set_title("Output")
                plt.imshow(images[i][0, 0, id, :, :], cmap='gray')
        interact(show_frame, 
                 id=widgets.IntSlider(min=0, max=images[0].shape[2]-1, step=1, value=images[0].shape[2]/2))
        
        
    def gen_patch_index(self, patch_size, patch_gap, index_path):
        count = 0
        patch_index = defaultdict(list)
        # https://arxiv.org/pdf/1710.02316.pdf
        # at least 0.01% voxels contain lesions
        voxel = int(patch_size[0]*patch_size[1]*patch_size[2]*0.0001)
        
        for i in self.data:
            if i == "patch_size":
                continue
            shape = self.data[i][0][0].shape
            patch_num = [int((shape[i]-patch_size[i]) / patch_gap) for i in range(len(shape))]

            for j in range(self.data[i].shape[0]):
                patch_ind = []
                # assume this is a 3d image
                for a in range(patch_num[0]):
                    for b in range(patch_num[1]):
                        for c in range(patch_num[2]):
                            patch_iter = [a * patch_gap, b * patch_gap, c * patch_gap, 1]
                            if (np.sum(self.data[i][j][1][patch_iter[0]:patch_iter[0] + patch_size[0],
                                                          patch_iter[1]:patch_iter[1] + patch_size[1],
                                                          patch_iter[2]:patch_iter[2] + patch_size[2]]) <= voxel):
                                # 0: does not satisfy, need to skip when generating
                                patch_iter[3] = 0
                            patch_ind.append(patch_iter)
                patch_index[i].append(patch_ind)
            
            for c in range(len(patch_index[i])):
                # in-place shuffle
                np.random.shuffle(patch_index[i][c])

            # total number of patches for this shape
            count += len(patch_ind) * self.data[i].shape[0]
        
        with h5py.File(index_path, 'w') as f:
            f.create_dataset("count", data=count)
            f.create_dataset("patch_size", data=patch_size)
            f.create_dataset("patch_gap", data=patch_gap)
            for i in patch_index:
                f.create_dataset(str(i), data=patch_index[i])
        
        index_file = h5py.File(index_path, 'r')
        for i in index_file.keys():
            if i == "count" or i == "patch_size" or i == "patch_gap":
                continue
            self.patch_index[i] = index_file[i][:]
        # return the total number of patches
        return index_file["count"][()]

    def load_patch_index(self, patch_size, patch_gap, index_path):
        if os.path.isfile(index_path):
            index_file = h5py.File(index_path, 'r')
            # print(list(pat_ind.keys()))
            if (np.all(index_file["patch_size"][:] == list(patch_size))) and (index_file["patch_gap"][()] == patch_gap):
                for i in index_file.keys():
                    if i == "count" or i == "patch_size" or i == "patch_gap":
                        continue
                    self.patch_index[i] = index_file[i][:]
                return index_file["count"][()]
            else:
                index_file.close()
                return self.gen_patch_index(patch_size, patch_gap, index_path)
        else:
            return self.gen_patch_index(patch_size, patch_gap, index_path)
            
    def prekfold(self, patch_size, patch_gap, batch_size, kfold=5, index_path='./model/h5df_data/pat_ind.h5'):
        self.kfold = kfold
        self.patch_size = patch_size
        self.patch_gap = patch_gap
        self.batch_size = batch_size

        # initialize validation index for training
        # K-fold LOOCV: leave one out cross validation
        for i in self.target:
            if i == "patch_size":
                continue
            self.valid_index[i] = random.sample(range(self.kfold), self.kfold)

        num = self.load_patch_index(patch_size, patch_gap, index_path)
        train_num = num // self.kfold * (self.kfold - 1)
        valid_num = num - train_num
        
        return train_num // batch_size, valid_num // batch_size
                    


In [3]:
import keras
import numpy as np

# https://stanford.edu/~shervine/blog/keras-how-to-generate-data-on-the-fly
# https://www.tensorflow.org/api_docs/python/tf/keras/utils/Sequence
class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, moda, pad_input, pad_target, pad_ind, kfold, batch_size, 
                 patch_size, patch_gap, valid_ind, is_train = False):
        self.moda = moda
        self.input = pad_input
        self.target = pad_target

        self.patch_index = pad_ind
        self.kfold = kfold
        self.batch_size = batch_size
        self.patch_size = patch_size
        self.patch_gap = patch_gap
        self.valid_index = valid_ind
        self.isTrain = is_train
        self.fold_index = None
        self.len = None
        
    def set_index(self, index):
        self.fold_index = index
        
    def __len__(self):
        if self.len == None:
            # count the total number of patches
            count = 0
            for i in self.patch_index:
                if i == "count" or i == "patch_size" or i == "patch_gap":
                    continue
                unit = self.patch_index[i].shape[0] // self.kfold
                iter_start = self.valid_index[i][self.fold_index] * unit
                iter_end = (self.valid_index[i][self.fold_index]+1) * unit
                for j in range(self.patch_index[i].shape[0]):
                    if j >= iter_start and j < iter_end:
                        if self.isTrain:
                            continue
                        else:
                            count += self.patch_index[i][j].shape[0]
                    else:
                        if self.isTrain:
                            count += self.patch_index[i][j].shape[0]
                        else:
                            continue
            self.len = count // self.batch_size
        return self.len
    
            
    def __getitem__(self, batch_index):
        start = batch_index * self.batch_size
        num = self.batch_size
        first_iter = True
        cur_image = []
        cur_target = []
        
        for i in self.patch_index:
            if i == "count" or i == "patch_size" or i == "patch_gap":
                continue
            unit = self.target[i].shape[0] // self.kfold
            iter_start = self.valid_index[i][self.fold_index] * unit
            iter_end = (self.valid_index[i][self.fold_index]+1) * unit
            for j in range(self.patch_index[i].shape[0]):
                # validation dataset: should validate on all patches
                if j >= iter_start and j < iter_end:
                    if self.isTrain:
                        continue
                    else:
                        if start > self.patch_index[i][j].shape[0]:
                            start -= self.patch_index[i][j].shape[0]
                        else:
                            # generate
                            if first_iter:
                                # first iter: append from start
                                start_iter = start
                                first_iter = False
                                for k in range(start_iter, self.patch_index[i][j].shape[0]):
                                    # if (patch[3] == 1): # should use all images for validation
                                    patch = self.patch_index[i][j][k]
                                    img = []
                                    for m in self.moda:
                                        image = self.input[m][i][j]
                                        img.append(image[patch[0]:patch[0]+self.patch_size[0], 
                                                         patch[1]:patch[1]+self.patch_size[1], 
                                                         patch[2]:patch[2]+self.patch_size[2]])
                                    target = self.target[i][j]
                                    cur_image.append(img)
                                    cur_target.append(target[patch[0]:patch[0]+self.patch_size[0], 
                                                             patch[1]:patch[1]+self.patch_size[1], 
                                                             patch[2]:patch[2]+self.patch_size[2]])
                                    num -= 1
                                    if (num == 0):
                                        return np.array(cur_image), np.expand_dims(cur_target, axis=1)
#                                         return np.expand_dims(cur_image, axis=1), np.expand_dims(cur_target, axis=1)
                            else:
                                # append from first patch for current image
                                for k in range(self.patch_index[i][j].shape[0]):
                                    # if (patch[3] == 1): # should use all images for validation
                                    patch = self.patch_index[i][j][k]
                                    img = []
                                    for m in self.moda:
                                        image = self.input[m][i][j]
                                        img.append(image[patch[0]:patch[0]+self.patch_size[0], 
                                                         patch[1]:patch[1]+self.patch_size[1], 
                                                         patch[2]:patch[2]+self.patch_size[2]])
                                    target = self.target[i][j]
                                    cur_image.append(img)
                                    cur_target.append(target[patch[0]:patch[0]+self.patch_size[0], 
                                                             patch[1]:patch[1]+self.patch_size[1], 
                                                             patch[2]:patch[2]+self.patch_size[2]])
                                    num -= 1
                                    if (num == 0):
                                        return np.array(cur_image), np.expand_dims(cur_target, axis=1)
#                                         return np.expand_dims(cur_image, axis=1), np.expand_dims(cur_target, axis=1)

                else:
                    # training dataset
                    if self.isTrain:
                        if start > self.patch_index[i][j].shape[0]:
                            start -= self.patch_index[i][j].shape[0]
                        else:
                            # generate
                            if first_iter:
                                # first iter: append from start
                                start_iter = start
                                first_iter = False
                                # feed some number of patches that does not have lesion
                                for k in range(start_iter, self.patch_index[i][j].shape[0]):
                                    patch = self.patch_index[i][j][k]
                                    if (patch[3] == 1):
                                        img = []
                                        for m in self.moda:
                                            image = self.input[m][i][j]
                                            img.append(image[patch[0]:patch[0]+self.patch_size[0], 
                                                             patch[1]:patch[1]+self.patch_size[1], 
                                                             patch[2]:patch[2]+self.patch_size[2]])
                                        target = self.target[i][j]
                                        cur_image.append(img)
                                        cur_target.append(target[patch[0]:patch[0]+self.patch_size[0], 
                                                                 patch[1]:patch[1]+self.patch_size[1], 
                                                                 patch[2]:patch[2]+self.patch_size[2]])
                                        num -= 1
                                        if (num == 0):
                                            return np.array(cur_image), np.expand_dims(cur_target, axis=1)
#                                             return np.expand_dims(cur_image, axis=1), np.expand_dims(cur_target, axis=1)
                            else:
                                # append from first patch for current image
                                for k in range(self.patch_index[i][j].shape[0]):
                                    patch = self.patch_index[i][j][k]
                                    if (patch[3] == 1):
                                        img = []
                                        for m in self.moda:
                                            image = self.input[m][i][j]
                                            img.append(image[patch[0]:patch[0]+self.patch_size[0], 
                                                             patch[1]:patch[1]+self.patch_size[1], 
                                                             patch[2]:patch[2]+self.patch_size[2]])
                                        target = self.target[i][j]
                                        cur_image.append(img)
                                        cur_target.append(target[patch[0]:patch[0]+self.patch_size[0], 
                                                                 patch[1]:patch[1]+self.patch_size[1], 
                                                                 patch[2]:patch[2]+self.patch_size[2]])
                                        num -= 1
                                        if (num == 0):
                                            return np.array(cur_image), np.expand_dims(cur_target, axis=1)
#                                             return np.expand_dims(cur_image, axis=1), np.expand_dims(cur_target, axis=1)
                    else:
                        continue



In [ ]:
import os

os.chdir("/scratch/yl4217/MS-Lesion-Segmentation/")

config = {}
config["weights_file"] = os.getcwd() + '/model/weight'
config["modality"] = ["FLAIR_preprocessed", "T1_preprocessed", "T2_preprocessed"]

config["patch_size"] = (64, 64, 64)  # switch to None to train on the whole image
config["patch_gap"] = 16
config["batch_size"] = 2
config["kfold"] = 5

config["input_shape"] = (len(config["modality"]), None, None, None)
config["depth"] = 4 # depth of layers for V/Unet
config["n_base_filters"] = 32
config["pool_size"] = (2, 2, 2)  # pool size for the max pooling operations
config["deconvolution"] = True  # if False, will use upsampling instead of deconvolution

config["patience"] = 10  # learning rate will be reduced after this many epochs if the validation loss is not improving
config["early_stop"] = 10  # training will be stopped after this many epochs without the validation loss improving
# config["initial_learning_rate"] = 0.00001
config["initial_learning_rate"] = 0.000001
config["learning_rate_drop"] = 0.5  # factor by which the learning rate will be reduced
config["n_epochs"] = 10


d = Data(config["modality"])
d.load_data(config["patch_size"])
train_num, valid_num = d.prekfold(config["patch_size"], config["patch_gap"], config["batch_size"], config["kfold"])

train_generator = DataGenerator(d.moda, d.input, d.target, d.patch_index, d.kfold, d.batch_size,
                                d.patch_size, d.patch_gap, d.valid_index, True)
valid_generator = DataGenerator(d.moda, d.input, d.target, d.patch_index, d.kfold, d.batch_size,
                                d.patch_size, d.patch_gap, d.valid_index, True)

print("multi-modality training on selected patch; dice loss")

def train(config, data, train_generator, valid_generator, train_num, valid_num):
#     models = []
    print(train_num, valid_num)
    for i in range(data.kfold):
        print ('-'*100)
        print ("Fold:", i)

        train_generator.set_index(i)
        valid_generator.set_index(i)

        model = unet_model_3d(input_shape=config["input_shape"],
                              pool_size=config["pool_size"],
                              initial_learning_rate=config["initial_learning_rate"],
                              deconvolution=config["deconvolution"],
                              depth=config["depth"],
                              n_base_filters=config["n_base_filters"])

        # model.load_weights(os.getcwd() + '/model/weight/weights-01-0.02-0428-binary-patch.hdf5')

        # print(model.summary())

        callbacks = get_callbacks(config["weights_file"], str(i)+'_dice_multimoda_',
                                initial_learning_rate=config["initial_learning_rate"],
                                learning_rate_drop=config["learning_rate_drop"],
                                learning_rate_patience=config["patience"],
                                early_stopping_patience=config["early_stop"])

        model.fit_generator(generator=train_generator,
                            steps_per_epoch=train_num,
                            epochs=config["n_epochs"],
                            validation_data=valid_generator,
                            validation_steps=valid_num,
                            callbacks=callbacks,
                            workers=2,
                            verbose=1)
        break
train(config, d, train_generator, valid_generator, train_num, valid_num)


multi-modality training on selected patch; dice loss
26496 6624
----------------------------------------------------------------------------------------------------
Fold: 0
Epoch 1/10
26496/26496 [==============================] - 15191s 573ms/step - loss: 0.5708 - dice_coefficient: 0.4292 - val_loss: 0.4932 - val_dice_coefficient: 0.5068
Epoch 2/10
  671/26496 [..............................] - ETA: 3:50:57 - loss: 0.4440 - dice_coefficient: 0.5560